In [1]:

###############################################################################
# Random Forest Classification Model (TensorFlow)                             #
# Based on the Implementation of:                                             #
# For GoldenEye Dataset                                                       #
# https://www.tensorflow.org/decision_forests/tutorials/beginner_colab        #
###############################################################################


In [ ]:
# Installieren aller benötigten Pakete
!pip install numpy==1.19.2
!pip install six==1.15.0
!pip install wheel==0.35
!pip install tensorflow_decision_forests
!pip install pandas
!pip install wurlitzer
!pip install matplotlib
!pip install ipython
!pip install dask
!pip install tf2onnx

In [7]:
# Laden der benötigten Python Pakete
import pandas as pd
import numpy as np
import tensorflow_decision_forests as tfdf
import tensorflow as tf
from wurlitzer import sys_pipes
import matplotlib.pyplot as plt
import dask.dataframe as dd
import tf2onnx

ModuleNotFoundError: No module named 'pandas'

In [9]:
# Prüfung der installierten TensorFlow Decision Forests Version
print(f"Found TensorFlow Decision Forests v{tfdf.__version__}")


NameError: name 'tfdf' is not defined

In [ ]:
# Laden der Netzwerk Traffic Daten für den GoldenEye Angriff
data_GoldenEye = pd.read_csv('../Data/Thursday-15-02-2018_GoldenEye-Attack.csv')
# Umbenennen der einzelnen Spalte für eine bessere Kompatibilität mit TensorFlow
data_GoldenEye.rename(columns={
    'Bwd Pkt Len Std':'bwd_pkt_len_std',
    'Flow IAT Min':'flow_iat_min',
    'Fwd IAT Min':'fwd_iat_min',
    'Flow IAT Mean':'flow_iat_mean',
    'Label':'label'
},
inplace=True)

In [ ]:
# Festlegen des Wertes der bestimmten Variable
label = 'label'

In [ ]:
# Aufteilen des Datasets in Training- und Test-Daten
def split_dataset(dataset,  test_ratio=0.30):
    """Splits a panda dataframe in two dataframes."""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

training_data_GoldenEye, testing_data_GoldenEye = split_dataset(data_GoldenEye)

print("{} examples in training, {} examples for testing.".format(
    len(training_data_GoldenEye), len(testing_data_GoldenEye)))


In [ ]:
# Konvertieren des Panda Dataframes in ein TensorFlow Dataset
print("Converting Panda Dataframe into TensorFlow Dataset...")
training_dataset_GoldenEye = tfdf.keras.pd_dataframe_to_tf_dataset(training_data_GoldenEye, label=label)
testing_dataset_GoldenEye = tfdf.keras.pd_dataframe_to_tf_dataset(testing_data_GoldenEye, label=label)


In [ ]:
# Erstellen des Random Forest Modells
model = tfdf.keras.RandomForestModel()
model.compile(metrics=["accuracy"])


In [ ]:
# Trainieren des Modells
print("Training the Model: ")
with sys_pipes():
    model.fit(x=training_dataset_GoldenEye)

In [ ]:
# Evaluation des trainierten Modells mit den Testdaten
print("Evaluating the Model...")
evaluation = model.evaluate(testing_dataset_GoldenEye, return_dict=True)

print()

for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")

In [ ]:
# Trainiertes Modell für die spätere Verwendung abspeichern
model.save("../Data/Models/goldeneye_model")


In [ ]:
# Plotten des ersten Baumes innerhalb des Decision Forests
with open('../Data/Models/GoldenEye_Model_Tree.html', 'w') as f:
    f.write(tfdf.model_plotter.plot_model(model, tree_idx=0, max_depth=3))
tfdf.model_plotter.plot_model(model, tree_idx=0, max_depth=3)


In [ ]:
# Erstellen einer Bilanz für das trainierte Modell
model.summary()

In [ ]:
# Erstellen von Grafiken für die Effizienz des Trainings
logs = model.make_inspector().training_logs()
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1,2,2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.savefig('../Data/Visualized/GoldenEye_Model.png')
plt.clf()